In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import os
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
import xgboost
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Importing User Defined modules
os.chdir(r'/kaggle/input/solar-eda')
import ads_creation as ads_crtn
import eda_analysis as eda
import feature_engg as feat_engg
import winsorize as wz
import regression as regr
import clustering as clst
from constants import ATTR, OUTLIER_METHOD, TIME_INVARIANT_ATTR, TIME_VARIANT_ATTR, SPLIT_PCT, IDENTIFIERS

<h1> Helper Functions </h1>

In [ ]:
#Funciton to create list of timestamps from first to last
def create_ts_list(df,date_col):
    '''
    This function creates the list of timestamps from first ts to last ts

    Input:
    1. df: pandas dataframe, this is the ads subsetted for one panel
    2. date_col: str, this is the name of the column containing the dates

    Return
    1. dates: pandas dataframe, containing all the timestamps from start to end
    '''
    dates = pd.date_range(start=df[date_col].min(), end = df[date_col].max(),freq = '15min')
    dates = dates.to_frame(index=False)
    dates = dates.rename(columns = {0:'DATE'})
    return dates


In [ ]:
#Function to merge in the missing dates in the ADS
def complt_ts(df,date_col):
    '''
    This function introduces the missing time stamps in ads

    Input:
    1. df: pandas dataframe, this is the ads subsetted for one panel
    2. date_col: str, this is the name of the column containing the dates

    Return
    1. df: pandas dataframe, containing all the timestamps from start to end
    '''
    date_list = create_ts_list(df,date_col)
    df = pd.merge(left=date_list,right=df,how='left', on='DATE')
    return df


In [ ]:
#Performing data completions at a panel level
def complt_ts_panel_lvl(df,panel_id_col,date_col):
    '''
    This function completes the time stamps for all panels and applies MVT as well

    Input:
    1. df: pandas dataframe, this is the ads for which timestamps need to be completed
    2. panel_id_col: str, the name of the column containing the panel IDs
    3. date_col, str, the name of the column containing the dates

    Returns
    1. df: pandas dataframe, containing all the timestamps from start to end
    '''
    df = df.groupby(panel_id_col).apply(lambda x: complt_ts(x, date_col))
    df = df.reset_index(drop=True)
    df['TIME_OF_DAY'] = df['DATE'].apply(lambda x: 1 if x.hour in range(6,18) else 0)
    #if we dont do this ffill here then the all the obs for inverter ids won't reach
    df[panel_id_col] = df[panel_id_col].ffill()
    df = df.groupby(panel_id_col).apply(lambda x: ads_crtn.missing_value_treatment(x))
    return df


In [ ]:
#Function to try different lags, starting from 2 days out till 2 hours and check the accuracy
def get_optimal_lag(train_df,init_lag,pass_threshold,mode,panel_id_col,model_obj,feature_selection=False):
    '''
    This function gets the lag that is most optimal in predicting the target variable
    Either in isolated mode or cumulative mode

    Input:
    1. train_df: pandas dataframe, this is the ads
    2. init_lag: int, the largest lag to begin with
    3. pass_threshold: float, the minimum r-squared values required
    4. mode: str, isolated or cumulative mode
    - Isolated mode: just creates one set of lags and check in silo
    - Cumulative mode: creates a group of lags like 2 day and 1 day lagged variables and checks accuracy together

    Return:
    1. model_dict: python dictionary, this dict contains model objects for each panel
    2. metric_dict: python dictionary, this dict contains error metrics for each panel
    '''

    #Creating a validation df because here we will be choosing the optimal lag
    #this is a form of tuning and test dataset should not be used
    train_df,validation_df = eda.train_test_split(train_df,'INVERTER_ID',0.9)
    r2 = 0
    lag = init_lag
    lag_list = [init_lag]

    #while loop runs till the time the condition is true
    #here we want the loop to stop as soon as one condition is met and the condition to become false
    #if we do or between the two conditions, while loop will still keep running because 0 + 1 = 1 and it will keep running
    #but if we put 'and' then 0 * 1 = 0 and we get the desired result
    while (r2 < pass_threshold) and (lag > 0):
        print('Doing for lag: ',lag)
        train_df_lagged = feat_engg.create_lagged_features(train_df,TIME_VARIANT_ATTR,lag_list,'INVERTER_ID','DATE')
        validation_df_lagged = feat_engg.create_lagged_features(validation_df,TIME_VARIANT_ATTR,lag_list,'INVERTER_ID','DATE')

        features = [feature for feature in train_df_lagged.columns if '_lag_' in feature]
        features = features + TIME_INVARIANT_ATTR
        
        if feature_selection == True:
            features = feat_engg.rfe_feature_selection(train_df_lagged,
                                                RandomForestRegressor(random_state=42),
                                                features,'PER_TS_YIELD')

        models,metrics = regr.panel_wise_model(train_df_lagged,validation_df_lagged,panel_id_col,'PER_TS_YIELD',
                                              model_obj,features=features)
        r2 = metrics[train_df_lagged[panel_id_col].unique()[0]][3]
        print(r2)

        if mode == 'isolated':
            lag -= 8
            lag_list = [lag]
        elif mode == 'cumulative':
            lag -= 8
            lag_list.append(lag)
    return models,metrics


In [ ]:
#Perseverence model
def fwd_step_rf_model(train_df,test_df,model_dict,panel_name):
    '''
    This function implements a perseverence model i.e. predicts 15 min in advance then uses that prediction 
    to churn out one more prediction

    Input
    1. train_df: pandas dataframe, training dataset
    2. test_df: pandas dataframe, testing dataset
    3. model_dict: dictionary, dict containing the prediciton models
    4. panel_name: str, string containing panel name for which the model will be run

    Returns:
    1. predicitions: list, which contains the predicitons made for 2 days
    '''
    predictions = []
    scaler = StandardScaler()
    scaler.fit(train_df.values.reshape(-1,1))
    train_df_scaled = scaler.transform(train_df.values.reshape(-1,1))
    pred_obs = train_df_scaled[-1]
    
    for iter in range(len(test_df)):
        if iter == 0:
            pred_obs = model_dict[panel_name].predict(pred_obs.reshape(-1,1))
            predictions.append(pred_obs)
        else:
            pred_obs = model_dict[panel_name].predict(scaler.transform(pred_obs.reshape(-1,1)))
            predictions.append(pred_obs)

    return predictions


In [ ]:
#Acutals vs Forecast
def act_vs_fcst(y_true,y_pred,date_series):
    '''
    This function plots actuals vs forecasts

    Input
    1. y_true: pandas series, ground truth
    2. y_pred: pandas series, predictions
    3. date_series: pandas series, series with the dates
    '''
    actual_df = pd.DataFrame({'DATE':date_series, 'Panel':'Actual',
                        'Actual':y_true})
    pred_df = pd.DataFrame({'DATE':date_series, 'Panel':'Predicted',
                        'Predicted':y_pred})

    plot_df = pd.concat([actual_df,pred_df])
    fig = px.line(plot_df,'DATE',['Actual','Predicted'])
    fig.show()

    return


In [ ]:
#Function for cleaning data
def remove_anomaly(df):
    '''
    This function removes instances wherein we have irradiation but AC power is 0

    Input:
    1. df: pandas dataframe, dataset containing noise

    Returns:
    df: pandas dataframe, dataset that is clean
    '''
    anam_idx = df.index[(df['IRRADIATION'] != 0) & (df['AC_POWER'] == 0)]
    df = df.drop(anam_idx)
    return df


<h1> Common steps for all Experiments </h1>

In [ ]:
#ADS creation, train-test split, outlier treatment
ads = ads_crtn.create_ads()
train_ads, test_ads = eda.train_test_split(ads,'INVERTER_ID',SPLIT_PCT)

#leaving out total yield since it value since it value increase exponentially and rightly so
outlier_feature = [feature for feature in ATTR if feature != 'TOTAL_YIELD'] 
clip_model = wz.winsorize(OUTLIER_METHOD)
clip_model.fit(train_ads[outlier_feature],'INVERTER_ID')
train_ads[outlier_feature] = clip_model.transform(train_ads[outlier_feature])
test_ads[outlier_feature] = clip_model.transform(test_ads[outlier_feature])


<h1> Experiment 1 </h1>

<h2> In this experiment, We will create lagged versions of all our features, wherein all the features would be lagged be 2 days. This way we can use the observations from today and predict 2 days in advance </h2>

In [ ]:
#Completing timestamps, so that each panel has all the timestamps from start to finish
train_ads_exp = complt_ts_panel_lvl(train_ads,'INVERTER_ID','DATE')
test_ads_exp = complt_ts_panel_lvl(test_ads,'INVERTER_ID','DATE')

#making ADS stationary
non_stnry_invtr_list = eda.return_non_stnry_invtr_list(train_ads_exp,'INVERTER_ID')
train_ads = eda.make_ads_stnry(train_ads_exp,non_stnry_invtr_list,'INVERTER_ID','DATE')

non_stnry_invtr_list = eda.return_non_stnry_invtr_list(test_ads_exp,'INVERTER_ID')
test_ads = eda.make_ads_stnry(test_ads_exp,non_stnry_invtr_list,'INVERTER_ID','DATE')

#Creating lagged features which are lagged by exactly 2 days so that they can then be used for prediction

train_ads_exp = feat_engg.create_lagged_features(train_ads_exp,TIME_VARIANT_ATTR,[192],'INVERTER_ID','DATE')
test_ads_exp = feat_engg.create_lagged_features(test_ads_exp, TIME_VARIANT_ATTR, [192], 'INVERTER_ID','DATE')

features = [feature + '_lag_192' for feature in TIME_VARIANT_ATTR]
features = features + TIME_INVARIANT_ATTR

models,metrics = regr.panel_wise_model(train_ads_exp,test_ads_exp,
                                        'INVERTER_ID','PER_TS_YIELD',
                                        RandomForestRegressor(random_state=42),
                                        features=features)


low_acc_invtr = [(key,metrics[key][3]) for key in metrics.keys() if metrics[key][3] < 0.8]
invtr_list = [val_pair[0] for val_pair in low_acc_invtr]
print(low_acc_invtr)


Format for reading the results: (Panel_Name:R squared value)

<h3> From the above experiment we saw that we are having a lot of trouble predicting for the plant 4136001 </h3>

<h1> Experiment 2 </h1>

<h2> Performing rfecv based on RF on the panels that gave poor accuracy in the previous step. So, that the significant variables may increase the accuracy </h2>

In [ ]:
metrics_rfe = {}
for invertor in invtr_list[0:2]:
    feat_set = feat_engg.rfe_feature_selection(train_ads_exp.loc[train_ads_exp['INVERTER_ID']==invertor,],
                                                RandomForestRegressor(random_state=42),
                                                features,'PER_TS_YIELD')
    
    models,metrics = regr.panel_wise_model(train_ads_exp.loc[train_ads_exp['INVERTER_ID']==invertor,],
                                            test_ads_exp.loc[test_ads_exp['INVERTER_ID']==invertor,],
                                            'INVERTER_ID',
                                            'PER_TS_YIELD',
                                            RandomForestRegressor(random_state=42),
                                            features=feat_set)
    print('Added metric for :',invertor)
    metrics_rfe[invertor] = metrics[invertor]

<h2> Performed this experiment on only 2 panels, for faster execution. Got very poor accuracy with rfecv as well </h2>

In [ ]:
print(metrics_rfe)

Format for reading the results: {Panel_name: (mape,mae,mse,r-squared)

<h1> Experiment 3 </h1>

<h2> Finding the most optimal lag starting from 2 days to 15 min. To further use in perseverence model </h2>

In [ ]:
#Finding the lag which gives the best accuracy, starting from 2 days out and moving closer to t-1
#All the different lags are looked at in isolation or cumulatively
#conducting this experiment with only one panel, if successful only then will scale up to other panels

trial_df = train_ads.loc[train_ads['INVERTER_ID']==invtr_list[1],]

#running it in isolated mode first
try_model,try_metric = get_optimal_lag(trial_df,192,0.8,'isolated',
                                        'INVERTER_ID',
                                        RandomForestRegressor(random_state=42),False)

#running the pipeline in cumulative mode
#better performance as compared to before but still did not cross 80% on validation dataset

#Commenting the below piece of code, since it takes a lot of time on local system
#try_model,try_metric = get_optimal_lag(trial_df,192,0.8,'cumulative')


<h2> Did not achieve good results with this as well, no lag was able to give more than 80% accuracy </h2>

<h1> Experiment 4 </h1>

<h2> Tried to get optimal lag with feature selection in cumulative mode & ran the code on google colab. It took a lot of time and did not get favorable results </h2>

<h1> Experiment 5 </h1>

<h2>Perseverence model</h2>
<h2>Forecast only 15 min in advance and use that forecast as input and forecast for another 15 min</h2>

In [ ]:
#Creating lagged version of target variable 
train_ads_exp = feat_engg.create_lagged_features(train_ads,['PER_TS_YIELD'],[1],'INVERTER_ID','DATE')
test_ads_exp = feat_engg.create_lagged_features(test_ads, ['PER_TS_YIELD'], [1], 'INVERTER_ID','DATE')

features = [feature for feature in train_ads_exp.columns if '_lag_' in feature]

models,metrics = regr.panel_wise_model(train_ads_exp,test_ads_exp,
                                        'INVERTER_ID','PER_TS_YIELD',
                                        RandomForestRegressor(random_state=42),features=features)

low_acc_invtr = [(key,metrics[key][3]) for key in metrics.keys() if metrics[key][3] < 0.8]
invtr_list = [val_pair[0] for val_pair in low_acc_invtr]

#Persevernce Model
fwd_models = {}
for invtr in invtr_list:
    print('Modelling for panel: ',invtr)
    preds = fwd_step_rf_model(train_ads_exp.loc[train_ads_exp['INVERTER_ID'] == invtr,features],
                            test_ads_exp.loc[test_ads_exp['INVERTER_ID'] == invtr,features],
                            models,invtr)


    y_pred = [prediction for sublist in preds for prediction in sublist]

    r2 = r2_score(test_ads_exp.loc[test_ads_exp['INVERTER_ID'] == invtr,'PER_TS_YIELD'],y_pred)
    fwd_models[invtr] = r2


In [ ]:
print(fwd_models['4UPUqMRk7TRMgml'])

<h2> Printed the accuracy for an example panel, like this panel most panels have negative r-squared. So this experiment also failed miserably </h2>

<h1> Experiment 6</h1>

<h2> Making a XGBoost model for all the panels that did not perform well in Experiment 1</h2>

In [ ]:
model = xgboost.XGBRegressor(random_state=42)

train_ads_exp_xgb = train_ads_exp.loc[train_ads_exp['INVERTER_ID'].isin(invtr_list),]
test_ads_exp_xgb = test_ads_exp.loc[test_ads_exp['INVERTER_ID'].isin(invtr_list),]
models_xgb,metrics_xgb = regr.panel_wise_model(train_ads_exp_xgb,test_ads_exp_xgb,'INVERTER_ID','PER_TS_YIELD',model,features=features)

low_acc_invtr = [(key,metrics_xgb[key][3]) for key in metrics_xgb.keys() if metrics_xgb[key][3] < 0.8]
invtr_list = [val_pair[0] for val_pair in low_acc_invtr]

In [ ]:
low_acc_invtr

Format for reading the result: (Panel_Name:R-squared)

<h2> XGBoost perfomed well, but still not 80% + accuracy</h2>

<h1> Experiment 7 </h1>

<h2> Making a model at a plant level, moving away from panel level models </h2>

In [ ]:
#Completing timestamps, so that each panel has all the timestamps from start to finish
train_ads_exp = complt_ts_panel_lvl(train_ads,'INVERTER_ID','DATE')
test_ads_exp = complt_ts_panel_lvl(test_ads,'INVERTER_ID','DATE')

train_ads_exp = feat_engg.create_lagged_features(train_ads_exp,TIME_VARIANT_ATTR,[192],'INVERTER_ID','DATE')
test_ads_exp = feat_engg.create_lagged_features(test_ads_exp, TIME_VARIANT_ATTR, [192], 'INVERTER_ID','DATE')

features = [feature + '_lag_192' for feature in TIME_VARIANT_ATTR]
features = features + TIME_INVARIANT_ATTR

models,metrics = regr.panel_wise_model(train_ads_exp,test_ads_exp,
                                        'PLANT_ID','PER_TS_YIELD',
                                        RandomForestRegressor(random_state=42),
                                        features=features)

In [ ]:
print(metrics)

Format for reading the results: {Plant name: (mape,mae,mse,r-squared)}

<h2> Getting good accuracy for plant 1, but very low for plant 2 </h2>

<h1> Experiment 8 </h1>

<h2>Clustering within plant 4136001 and making a model for each cluster</h2>

In [ ]:
train_ads_clst = train_ads.loc[train_ads['PLANT_ID']==4136001.0].reset_index(drop=True)
test_ads_clst = test_ads.loc[test_ads['PLANT_ID']==4136001.0].reset_index(drop=True)

#running the below loop to give us the number of clusters that are optimal for this clustering exercise
#10 clusters are coming out to be most optimal, so lets go ahead witthat

n_clust = clst.get_n_clusters(train_ads_clst.loc[:,~train_ads_clst.columns.isin(IDENTIFIERS)],2,11)

#kmeans clustering with 10 clusters
clusterer = KMeans(n_clusters=n_clust, random_state=42)
cluster_labels = clusterer.fit_predict(train_ads_clst.loc[:,~train_ads_clst.columns.isin(IDENTIFIERS)])

train_ads_clst = pd.concat([pd.DataFrame({'CLUSTER_ID':list(cluster_labels)}),train_ads_clst],axis=1)

#putting the cluster IDs in the test ads as well
invtr_clst_map = train_ads_clst[['INVERTER_ID','CLUSTER_ID']].drop_duplicates().reset_index(drop=True)
test_ads_clst = pd.merge(test_ads_clst,invtr_clst_map,on='INVERTER_ID',how='inner')



In [ ]:
#Making model for each cluster
train_ads_clst = feat_engg.create_lagged_features(train_ads_clst,TIME_VARIANT_ATTR,[192],'INVERTER_ID','DATE')
test_ads_clst = feat_engg.create_lagged_features(test_ads_clst, TIME_VARIANT_ATTR, [192], 'INVERTER_ID','DATE')

features = [feature + '_lag_192' for feature in TIME_VARIANT_ATTR]
features = features + TIME_INVARIANT_ATTR

models,metrics = regr.panel_wise_model(train_ads_clst,test_ads_clst,
                                        'CLUSTER_ID','PER_TS_YIELD',
                                        RandomForestRegressor(random_state=42),
                                        features=features)

In [ ]:
print(metrics)

<h2> Even one model for each cluster is not giving good accuracy, will have to move onto something else </h2>

<h1> Experiment 9 </h1>

<h2> Trying to find optimal lag while making a model at a plant level </h2>

In [ ]:
train_ads_exp = train_ads.loc[train_ads['PLANT_ID']==4136001.0].reset_index(drop=True)
test_ads_exp = test_ads.loc[test_ads['PLANT_ID']==4136001.0].reset_index(drop=True)

_,metrics = get_optimal_lag(train_ads_exp,192,0.8,'isolated','PLANT_ID',RandomForestRegressor(random_state=42))


<h2> No luck with different lags when modelling for the entire panel as well </h2>

<h1> Experiment 10 </h1>

<h2> Removing the noisy observations from the data, in which there is no AC power even when the Irradiation is non zero </h2>

In [ ]:
    train_ads_exp = remove_anomaly(train_ads)
    test_ads_exp = remove_anomaly(test_ads)

    train_ads_exp = complt_ts_panel_lvl(train_ads_exp,'INVERTER_ID','DATE')
    test_ads_exp = complt_ts_panel_lvl(test_ads_exp,'INVERTER_ID','DATE')

    train_ads_exp = feat_engg.create_lagged_features(train_ads_exp,TIME_VARIANT_ATTR,[192],'INVERTER_ID','DATE')
    test_ads_exp = feat_engg.create_lagged_features(test_ads_exp, TIME_VARIANT_ATTR, [192], 'INVERTER_ID','DATE')

    features = [feature + '_lag_192' for feature in TIME_VARIANT_ATTR]
    features = features + TIME_INVARIANT_ATTR + ['TIME_OF_DAY']
    
    models,metrics = regr.panel_wise_model(train_ads_exp,test_ads_exp,
                                        'PLANT_ID','PER_TS_YIELD',
                                        RandomForestRegressor(random_state=42),
                                        features=features)

In [ ]:
metrics

<h2> With this denoised dataset, our random forest model finally comes good and we get 80% + accuracy on both the data sets </h2>

<h2> Plotting Actuals vs Predicted for a panel from plant 4135001 </h2>

In [ ]:
scaler = StandardScaler()
scaler.fit(train_ads_exp.loc[train_ads_exp['PLANT_ID'] == 4135001,features])

train_df_scaled = scaler.transform(train_ads_exp.loc[train_ads_exp['PLANT_ID'] == 4135001,features])
test_df_scaled = scaler.transform(test_ads_exp.loc[test_ads_exp['PLANT_ID'] == 4135001, features])

panel_idx = test_ads_exp.index[test_ads_exp['INVERTER_ID'] == '1BY6WEcLGh8j5v7']

y_pred = models[4135001].predict(test_df_scaled[panel_idx])

act_vs_fcst(test_ads_exp.loc[panel_idx,'PER_TS_YIELD'], y_pred, test_ads_exp.loc[panel_idx,'DATE'])

<h2> As we can see the model is able to very well capture the variation present in the data </h2>

<h2> Plotting Actuals vs Predicted for a panel from plant 4136001 </h2>

In [ ]:
scaler = StandardScaler()
scaler.fit(train_ads_exp.loc[train_ads_exp['PLANT_ID'] == 4136001,features])

train_df_scaled = scaler.transform(train_ads_exp.loc[train_ads_exp['PLANT_ID'] == 4136001,features])
test_df_scaled = scaler.transform(test_ads_exp.loc[test_ads_exp['PLANT_ID'] == 4136001, features])

panel_idx = test_ads_exp.index[test_ads_exp['INVERTER_ID'] == 'Qf4GUc1pJu5T6c6']

y_pred = models[4136001].predict(test_df_scaled[panel_idx])

act_vs_fcst(test_ads_exp.loc[panel_idx,'PER_TS_YIELD'], y_pred, test_ads_exp.loc[panel_idx,'DATE'])

<h2> Here as well our model does well, but not quite as good as plant 1 model. We can try and squeze out more performance, but we will stop here for now </h2>